Import modules and check that version is correct.

In [1]:
import tensorflow as tf
import onnxruntime as ort
print("runtime", ort.__version__)
import onnx
print("onnx", onnx.__version__)
import onnxsim 
print("simplifier", onnxsim.__version__)
import onnx_graphsurgeon
print("surgeon", onnx_graphsurgeon.__version__)
import simple_onnx_processing_tools
print("processing tools",simple_onnx_processing_tools.__version__)
import tensorflow as tf
print("tensorflow",tf.__version__)
import tf_keras
print("tfkeras",tf_keras.__version__)
import ai_edge_litert
print("ai", ai_edge_litert.__version__)
import psutil
print("psutil", psutil.__version__)
import ml_dtypes
print("ml_dtypes", ml_dtypes.__version__)
import flatbuffers
print("flat", flatbuffers.__version__)
import onnx2tf

runtime 1.18.1
onnx 1.17.0
simplifier 0.4.33
surgeon 0.5.6
processing tools 1.1.32
tensorflow 2.17.0
tfkeras 2.19.0
ai 1.2.0
psutil 5.9.5
ml_dtypes 0.5.1
flat 24.3.25


Import onnx model and convert it into tensorflow using the onnx2tf module. To acquire the onnx NeuralHash model (the that we convert below) follows the steps in https://github.com/AsuharietYgvar/AppleNeuralHash2ONNX

In [ ]:
model_path="/Users/omaralmutoteh/NeuralHash/model.onnx"
tf_model=onnx2tf.convert(input_onnx_file_path=model_path)

Necessary functions

In [15]:
import numpy as np
from PIL import Image

def load_image_tf(path):
    im = Image.open(path).convert('RGB')
    im = im.resize([360, 360])
    arr = np.array(im).astype(np.float32) / 255.0
    arr = arr.reshape([1, 360, 360, 3])
    return arr

def save_image(arr, path):
    arr = arr.reshape([360, 360, 3]) # changes shape of array from (1, 360, 360, 3) to (360, 360, 3)
    arr = (arr) * (255.0)
    arr = arr.astype(np.uint8)
    im = Image.fromarray(arr)
    im.save(path)

def load_seed(path):
    seed = open(path, 'rb').read()[128:]
    seed = np.frombuffer(seed, dtype=np.float32)
    seed = seed.reshape([96, 128])
    return seed

def hash_from_hex(hex_repr):
    n = int(hex_repr, 16)
    h = np.zeros(96)
    for i in range(96):
        h[i] = (n >> (95 - i)) & 1
    return h

def hash_to_hex(h):
    bits = ''.join(['1' if i >= 0.5 else '0' for i in h])
    return '{:0{}x}'.format(int(bits, 2), len(bits) // 4)

In the cell below we go thru the method by which our tf2 keras model is converted into its serialzed form so that it can be deployed in a tf1 graph session. 

In [6]:
import tensorflow as tf
import numpy as np
import os

tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()

"""
Our main goal is to run a tf1 session which executes the graph version of 
tf_model.
Because our model is a tf2 keras model what we're demanding is a tough ask,  
since one can only convert tf_model into a tf2 graph which isn't executable
on tf1 sessions (due to tf2's management of a graph's content as variables,
which tf1 has no idea how to handle)
"""

"""
The first step is to just convert your model into a tf2 graph, or what is 
known as a static graph and this is done by tracing the model 
thru this decorator business
"""

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

"""
In the lines below we use the @ decorator to convert our tf_model func
into its graph equivalent. 
This only produces a template graph, to generate the version which u can execute on
we need to do the .get_concrete_function() call.
As it stands we now have our tf2 graph, which is a fine object. You will
be able to eagerly execute your model on inputs, and your tf2 graph will
do what its supposed to and execute in a graph way.
The problem is that you can't, as it stands, execute this graph in a tf1 
session.
"""

model = tf_model
@tf.function(input_signature=[tf.TensorSpec([1, 360, 360, 3], tf.float32)])
def model_func(x):
    return model(x)
concrete_func = model_func.get_concrete_function()

"""
To convert our graph so that we can run a tf1 sess on it,
we will produce a blank tf1 graph, work within it and then 
populate it with our tf2 graph's values.
To populate a blank graph with another graph we need serialize 
the latter graph using .graph.as_graph_def and then load it in
a tf1 session using tf.import_graph_def to get 
the unseralized version of the graph (i.e. our desired graph).
The convert_variables_to_constants_v2 call converts the 
values in your graph from type variables to type constants. 
We need to do this so that we get our graph to the point
whereby its eligible to be converted into its pb format (serialzed form) and 
so it's exportable to tf1.
"""

frozen_func = convert_variables_to_constants_v2(concrete_func)

"""
The below converts our graph into pb format.
"""

graph_def = frozen_func.graph.as_graph_def()

"""
Creates our blank graph.
"""

graph = tf.Graph()

I0000 00:00:1744546182.665960 6599105 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)


In the cell below we use the seralized form of our tf model to build its equivalent tf1 graph. Moreover we extend the graph so that upon running a session on it we are able to achieve values which are required for our ultimate on the cat image.

In [55]:
"""
The with loop below allows us to work with our 
blank tf1 graph.
"""

with graph.as_default():

    """
    We will build our desired graph by populating 
    our blank graph with our tf2 graph using the pb version of the tf2 graph.
    This is done thru the tf.import_graph_def call below.
    """
    
    tf.import_graph_def(graph_def, name="")

    """
    Now that we have the tf1 graph equiavlent of our model, we refer to 
    the input and output of our model by the name of the "nodes" in 
    the tf1 graph which correspond to those objects.
    Moreover, when we define a new variable in terms of an old one (such 
    as here: logits = tf.reshape(logits_raw, (128,))) we are effectively 
    creating a new node within the graph which we can then recover 
    upon the execution of the graph (in the same way that we do to 
    get, say, the output node.
    Finally, we can also create new nodes which don't have to be in terms of old 
    nodes thru this "placeholder" method. We want to do this so that we can 
    input in fixed values when we run the session, and this is done by 
    specifying the placeholder value.
    Why are we creating new nodes?
    Our attack on the original image is a gradient based one, thus we need to 
    be able to produce the gradients to alter the image in the correct direction.
    Thus, gradients become a new node in our graph whose values can now be attained.
    """
    
    # From frozen graph
    image_ph = graph.get_tensor_by_name("x:0")              # input name is "x:0"
    logits_raw = graph.get_tensor_by_name("Identity:0")     # output name is "Identity:0"
    logits = tf.reshape(logits_raw, (128,))                # need to reshape because of the unnecessary dims

    # Our placeholders
    original_ph = tf.compat.v1.placeholder(tf.float32, shape=(1, 360, 360, 3), name="original")
    seed_ph = tf.compat.v1.placeholder(tf.float32, shape=(96, 128), name="seed")
    h_ph = tf.compat.v1.placeholder(tf.float32, shape=(96,), name="h")


    # Hashing pipeline
    proj = tf.reshape(tf.matmul(seed_ph, tf.reshape(logits, (128, 1))), (96,))

    """
    We want to L2 normalize the output 96D vector because the following operation is 
    the sigmoid function which is 0,1 for large negative and positive numbers, respectively.
    This means that without normalize, the hash_output node is effectively a 0,1 constant 
    function about negative and non-negative values. 
    Thus, its gradients will be 0.
    Hence, we need to normalize the valyes of proj prior to entering them into sigmoid,
    so that we don't get this 0,1 effect.
    """
    
    normalized, _ = tf.linalg.normalize(proj)

    """
    We need to use the sigmoid func in place of 
    the neg, non-neg thresholding (which is normally used to achieve hash).
    This is because the latter is not differentiable.
    """
    
    hash_output = tf.sigmoid(normalized)

    # Losses
    hash_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(h_ph, hash_output))

    perturbation = image_ph - original_ph
    img_loss = 0.99* tf.nn.l2_loss(perturbation) + 0.01* tf.image.total_variation(perturbation)[0]

    combined_loss = 0.8*hash_loss + 0.2*img_loss

    
    # Gradients

    """
    When calculating our desired gradients we ought to differentiate the losses wrt the input,
    i.e. the image. 
    This is because we want to change our image until its respective loss is min.
    """
    
    g_hash = tf.gradients(hash_loss, image_ph)[0]
    g_img, = tf.gradients(img_loss, image_ph)
    g_combined, = tf.gradients(combined_loss, image_ph)

In the cell below we go about loading the input values that we will use in when we run a session on our graph. For example it's here that we specify what the target hash's bits are, so that we can input it in the feed dictionary for the key h_ph. 

In [61]:
# === Load inputs ===
path_to_cat = "/Users/omaralmutoteh/Desktop/GitHub/neural-hash-collider/cat.png"
seed_path = "/Users/omaralmutoteh/NeuralHash/neuralhash_128x96_seed1.dat"
original_np = load_image_tf(path_to_cat)          # shape (1, 360, 360, 3)
seed_np = load_seed(seed_path)           # shape (96, 128)
h_np = hash_from_hex('59a34eabe31910abfb06f308')   # target hash's bits
x = np.copy(original_np)
best = 100  # (distance, image quality loss)
dist = 96

It's in the cell below where we ultimately run our attack on the cat image.

In [62]:
with tf.compat.v1.Session(graph=graph) as sess:
    for i in range(5000):

        """
        Our attack works as follows:
        We initially aim to just alter the image until a collison is found, 
        that is the new image's bits are the same as h_np. 
        Once that's achieved choose to minimize the difference 
        between the new image and the original cat image. 
        However this is done with the intention that the 
        changes made upon the image ensure that the difference 
        in the bits between the hash of the new image and h_np is
        at most 2
        This ensures that our image doesn't stray too far from 
        a collision.
        """


        """
        The first if's condition is achieve our current image is a collison,
        once that happens the new gradient that we apply the GD method 
        on is that of the img loss.
        This is done so that we start changing our image to minimize 
        the distance between the current image and the original 
        cat image.
        The reason this is done is because once our image is a collison
        all we care about should be to minimize image perturbation
        """
        if dist == 0:
            loss_name, loss, g = 'image', img_loss, g_img
        
        
        #The next if condition is achieved when a collision is achieved
        #in some iteration in the past (current image need not be collison).
        #If that occurs then we want to check whether our current image's
        #hash is close to the target, and we define close as being 2 bits off.
        #If it is close then we can afford to minimize both hash_loss and 
        #img_loss without being worried that our has is going to stary away too far
        #as it's already close.
        
        
        elif ((best == 0) and (dist <= 2) and (dist > 0)):
            loss_name, loss, g = 'combined', combined_loss, g_combined


        
        #The only alternative left is a collision not having been found, or that 
        #one has been found but that we've strayed far from the target bits.
        #In that case, we ask our algorithm to alter the img in the direction
        #whereby hash_loss is minimized to get our image's hash back to being 
        #close.
        
        else:
            loss_name, loss, g = 'hash', hash_loss, g_hash

        
        """
        The next if is made because by iteration 2500 I found that a collision will
        have already been found plus the img perturbation will be desirable.
        Thus, we just need to ensure that the img stays on track in terms of 
        being a collision.
        Thus, we ask our algo to minimize for hash_loss.
        """
        
        
        if i > 2500:
            loss_name, loss, g = 'hash', hash_loss, g_hash
            

        """
        As I've mentioned before, for us to run a session on our tf1 graph,
        we need to supply the session with the node values for it to complete
        a session whereby all output nodes' values can be generated.
        Thus, the following dictionary is filled to capture all of that.
        """

        feed = {
            image_ph: x,
            original_ph: original_np,
            seed_ph: seed_np,
            h_ph: h_np,     
        }


        """
        The line below is where we run our session on our graph,
        moreover in the 1st argument we specify a list of values 
        and these correspond to all the nodes' values which we want
        generated.
        """
        hash_output1, img_loss1, loss1, g1  = sess.run(
            [hash_output, img_loss, loss, g],
            feed_dict=feed
        )

        """
        In the next 3 lines we compute the dist value corresponding 
        to the current img. 
        This is simply applying <=0.5 and >0.5 thresholding on its
        hash_output, that is on the output of the sigmoid function.
        """

        binary_hash = (hash_output1 > 0.5).astype(np.float32)
        match = np.sum(binary_hash == h_np)
        dist = 96 - match

        """
        Once we have our dist, we can now check if our current
        img is a collision.
        If this happens to be the case with the number of iterations also being 
        greater than 2500 then we can break our algorithm due to the 
        aforementioned reasons.
        """

        if i > 2500 and dist == 0:

            break


        if dist < best:

            best = dist


        """
        The gradient we want to use in the GD step is the normalized 
        version of our generated gradient since I've found that 
        the gradient values tend to be small in scale (same 
        reasoning as when we used normalization to generate 
        the normalized node in the graph construction cell).
        """

        g1 /= (np.linalg.norm(g1)+ 1e-8)
        x = x - 0.6 * g1

        """
        We apply this clip step so that our img doesn't stray to 
        far from the previous img.
        """
        x = np.clip(x, 0.0, 1.0)
    
        if i%10==0:
            
            print("iteration: {}, best: {}, dist: {}, loss: {}".format(

                i, best, dist, loss_name
            ))


iteration: 0, best: 50, dist: 50, loss: hash
iteration: 10, best: 40, dist: 40, loss: hash
iteration: 20, best: 39, dist: 39, loss: hash
iteration: 30, best: 35, dist: 35, loss: hash
iteration: 40, best: 28, dist: 28, loss: hash
iteration: 50, best: 25, dist: 26, loss: hash
iteration: 60, best: 24, dist: 24, loss: hash
iteration: 70, best: 22, dist: 24, loss: hash
iteration: 80, best: 21, dist: 22, loss: hash
iteration: 90, best: 19, dist: 19, loss: hash
iteration: 100, best: 17, dist: 20, loss: hash
iteration: 110, best: 17, dist: 17, loss: hash
iteration: 120, best: 15, dist: 15, loss: hash
iteration: 130, best: 13, dist: 14, loss: hash
iteration: 140, best: 12, dist: 12, loss: hash
iteration: 150, best: 12, dist: 13, loss: hash
iteration: 160, best: 12, dist: 13, loss: hash
iteration: 170, best: 12, dist: 13, loss: hash
iteration: 180, best: 12, dist: 13, loss: hash
iteration: 190, best: 12, dist: 12, loss: hash
iteration: 200, best: 11, dist: 11, loss: hash
iteration: 210, best: 11

In [64]:
save_image(x, os.path.join('.', "cat_attack.png"))